# Offline Test: CDI Generation (No Service)

This notebook calls `generate_cdi` directly from the repo to build a CDI graph without running the FastAPI service.


In [11]:
import os
import sys
import json
import rdflib
from pprint import pprint
from typing import Optional


def generate_cdi_offline(source_url: str, export_path: str, export_format: str, resources_dir: Optional[str], dataset_type: Optional[str], datasetid: Optional[str] = None, datasetversion: Optional[str] = None) -> None:
    resolve_api_path()
    from cdi import CDI_DDI

    resources_dir_final = resources_dir or os.path.join(os.path.dirname(os.path.abspath(__file__)), "resources")
    generator = CDI_DDI(
        url=source_url,
        export_file=export_path,
        export_format=export_format,
        resources_dir=resources_dir_final,
        type=dataset_type,
    )
    cdi_graph = generator.parse_cdi()
    # Quick sanity/compatibility check of the CDI graph
    try:
        _ = cdi_graph.serialize(format="json-ld")
    except Exception:
        print("Warning: CDI graph serialization failed; using empty graph as fallback.")
        cdi_graph = rdflib.Graph()
    # Also enrich the graph with schema.org JSON-LD from Dataverse
    schema_url = None
    if datasetid:
        # Try to infer base site from source_url; fallback to dev codata
        try:
            from urlparse import urlparse  # Python2 compat if needed
        except Exception:
            from urllib.parse import urlparse
        try:
            parsed = urlparse(source_url)
            base = parsed.scheme + "://" + parsed.netloc if parsed.scheme and parsed.netloc else "https://dataverse.dev.codata.org"
        except Exception:
            base = "https://dataverse.dev.codata.org"
        schema_url = base.rstrip("/") + "/api/datasets/export?exporter=schema.org&persistentId=" + datasetid
    else:
        # Default example fallback if no datasetid provided
        schema_url = "https://dataverse.dev.codata.org/api/datasets/export?exporter=schema.org&persistentId=doi%3A10.5072/FK2/8MODGT"
    try:
        schema_graph = rdflib.Graph()
        schema_graph.parse(schema_url, format="json-ld")
        # Merge schema_graph into cdi_graph
        for triple in schema_graph:
            cdi_graph.add(triple)
    except Exception:
        # Ignore enrichment errors to not block core generation
        pass
    if export_path and export_format:
        if export_format == "json-ld":
            raw_jsonld = cdi_graph.serialize(format="json-ld")
            try:
                parsed = json.loads(raw_jsonld)
            except Exception:
                parsed = {"@graph": []}
            graph_nodes = parsed.get("@graph", parsed if isinstance(parsed, list) else [parsed])
            wrapped = {
                "@graph": graph_nodes
            }
            with open(export_path, "w") as f:
                f.write(json.dumps(wrapped, indent=2, ensure_ascii=False))
        else:
            cdi_graph.serialize(destination=export_path, format=export_format)
        print("Exported CDI graph to %s (%s)" % (export_path, export_format))
    return cdi_graph


# Ensure repo root is in sys.path (parent of notebooks/)
REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

# Ensure api/ is importable (cdi_generator resolves it too)
API_DIR = os.path.join(REPO_ROOT, "api")
if API_DIR not in sys.path:
    sys.path.insert(0, API_DIR)

print("Repo root:", REPO_ROOT)
print("API dir:", API_DIR)


Repo root: /Users/vyacheslavtykhonov/projects/cdi-xas
API dir: /Users/vyacheslavtykhonov/projects/cdi-xas/api


In [12]:
from typing import Optional


In [ ]:
# Define a wrapper that reuses repository generate_cdi (no resolve_api_path here)
from cdi_generator import generate_cdi as _generate_cdi

def generate_cdi_offline(source_url: str,
                         export_path: str,
                         export_format: str,
                         resources_dir: Optional[str],
                         dataset_type: Optional[str],
                         datasetid: Optional[str] = None,
                         datasetversion: Optional[str] = None):
    return _generate_cdi(source_url, export_path, export_format, resources_dir, dataset_type, datasetid, datasetversion)


In [13]:
from cdi_generator import generate_cdi

# Example inputs (adjust as needed)
SITE_URL = os.environ.get("DV_SITE", "https://dataverse.dev.codata.org")
FILE_ID = os.environ.get("DV_FILEID", "25")
PERSISTENT_ID = os.environ.get("DV_PERSISTENT_ID", "doi:10.5072/FK2/4ZSKVU")

# Build direct datafile access URL
source_url = f"{SITE_URL.rstrip('/')}/api/access/datafile/{FILE_ID}"
print("Source URL:", source_url)
print("Persistent ID:", PERSISTENT_ID)


Source URL: https://dataverse.dev.codata.org/api/access/datafile/25
Persistent ID: doi:10.5072/FK2/4ZSKVU


In [14]:
# Generate CDI graph directly (no service)
# export_path=None to avoid writing; export_format used internally
cdi_graph = generate_cdi_offline(
    source_url=source_url,
    export_path=None,
    export_format="json-ld",
    resources_dir=None,
    dataset_type="xas",
    datasetid=PERSISTENT_ID,
)

print("Triples in CDI graph:", len(cdi_graph))


NameError: name 'resolve_api_path' is not defined

In [ ]:
# Ensure a CDI graph is available, then flatten JSON-LD with pyld and preview
#!pip install pyld
try:
    _ = cdi_graph  # check if defined
except NameError:
    import os, sys
    # Make sure repo root is importable (parent of notebooks/)
    REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
    if REPO_ROOT not in sys.path:
        sys.path.insert(0, REPO_ROOT)
    from cdi_generator import generate_cdi
    SITE_URL = os.environ.get("DV_SITE", "https://dataverse.dev.codata.org")
    FILE_ID = os.environ.get("DV_FILEID", "25")
    PERSISTENT_ID = os.environ.get("DV_PERSISTENT_ID", "doi:10.5072/FK2/4ZSKVU")
    source_url = f"{SITE_URL.rstrip('/')}/api/access/datafile/{FILE_ID}"
    cdi_graph = generate_cdi(
        source_url=source_url,
        export_path=None,
        export_format="json-ld",
        resources_dir=None,
        dataset_type="xas",
        datasetid=PERSISTENT_ID,
    )

jsonld_str = cdi_graph.serialize(format="json-ld")
if jsonld_str:
    from pyld import jsonld as jsonldlib
    doc = json.loads(jsonld_str)
    flattened = jsonldlib.flatten(doc)
    preview = json.dumps(flattened, indent=2, ensure_ascii=False)

print("Flattened JSON-LD preview (first 2000 chars):\n")
print(preview)


Flattened JSON-LD preview (first 2000 chars):

[
  {
    "@id": "_:b0",
    "http://www.w3.org/2004/02/skos/core#definition": [
      {
        "@value": "Si(111)"
      }
    ]
  },
  {
    "@id": "_:b1",
    "http://www.w3.org/2004/02/skos/core#definition": [
      {
        "@value": "CC0-4.0"
      }
    ]
  },
  {
    "@id": "_:b10",
    "http://www.w3.org/2004/02/skos/core#definition": [
      {
        "@value": "none"
      }
    ]
  },
  {
    "@id": "_:b11",
    "http://www.w3.org/2004/02/skos/core#definition": [
      {
        "@value": "r.t."
      }
    ]
  },
  {
    "@id": "_:b12",
    "http://www.w3.org/2004/02/skos/core#definition": [
      {
        "@value": "Energy"
      }
    ]
  },
  {
    "@id": "_:b13",
    "http://www.w3.org/2004/02/skos/core#definition": [
      {
        "@value": "I0"
      }
    ]
  },
  {
    "@id": "_:b14",
    "http://www.w3.org/2004/02/skos/core#definition": [
      {
        "@value": "Itrans"
      }
    ]
  },
  {
    "@id": "_:b15

In [ ]:
# Optionally write to a file under resources for inspection
OUT = os.environ.get("OUT_JSON", "resources/offline_cdi.jsonld")
with open(os.path.join(REPO_ROOT, OUT), "w") as f:
    f.write(jsonld_str)
print("Wrote:", os.path.join(REPO_ROOT, OUT))


Wrote: /Users/vyacheslavtykhonov/projects/cdi-xas/resources/offline_cdi.jsonld
